In [ ]:
import sys
import os
from pprint import pprint


current_dir = os.path.dirname(os.path.realpath(os.getcwd()))
different_path = os.path.abspath(os.path.join(current_dir, '../'))
sys.path.append(current_dir)

from modules.graphs import build_graph, graph_names
from modules.query_resolver import query_resolver
from modules.queries import queries

In [ ]:
import psycopg2
import psycopg2.extras

from urllib.parse import urlparse
import os
from decouple import AutoConfig
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
config = AutoConfig(search_path=parent_directory)
url = urlparse(config("db_url"))
connection = psycopg2.connect(
    host=url.hostname,
    port=url.port,
    database=url.path[1:],
    user=url.username,
    password=url.password
)
cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

In [ ]:
result = query_resolver(cursor, queries, "list_guilds")
guild_id = result.iloc[0]["guild_id"]
print(f"guild_id = {guild_id}")

In [ ]:
result = query_resolver(cursor, queries, "guild_author_most_reactions", {"guild_id" : guild_id })
author_id = result.iloc[0]["author_guild_id"]
print(f"author_id = {author_id}")

In [ ]:
for graph_name in list(graph_names.keys()):#[7:]:
    print(graph_name)
    fig = build_graph(
        cursor,
        graph_name,
        {
            "guild_id" : guild_id,
            "author_id": author_id
        }
    )
    if type(fig) == type("") or fig == None:
        pprint(fig)
    fig["fig"].update_layout(fig["layout"])
    fig["fig"].show()